## Contrastive Unlearning by Hong Kyu Lee et al at IJCAI 2025

Paper link - https://arxiv.org/pdf/2401.10458
1. They propose contrastive learning inspired unlearning paradigm. which utilises the concept of anchor, poisitive and negative samples.
2. They show perfect forget accuracy and good retain accuracy on the emprical results.
3. This works for both class level and sample level unlearning

#### Lacks:
1. Broader testing of model types, study focuses only on ResNet architecture family.
2. Limited variety of datasets, as the authors have trained on only CIFAR10 and SVHN.

In [39]:
!pip install torchvision torch scikit-learn -q

In [5]:
from torchvision.models import resnet18
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import copy
import random
import os
from torch import optim

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18

class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        backbone = resnet18(pretrained=True)
        
        # Remove the final FC layer
        self.feature_extractor = nn.Sequential(*list(backbone.children())[:-1])  # Output: (B, 512, 1, 1)
        
        self.fc = nn.Linear(512, num_classes)  # 512 is the output channels of last conv block

    def forward(self, x):
        feats = self.feature_extractor(x)           # (B, 512, 1, 1)
        feats = feats.view(feats.size(0), -1)       # Flatten to (B, 512)
        norm_feats = F.normalize(feats, p=2, dim=1) # L2 normalize
        logits = self.fc(norm_feats)                # Class scores

        return logits, norm_feats

In [7]:
### Hyperparameters and Dataset Loading

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 32
epochs = 5  # Short for demo; increase for better accuracy
lr = 0.1
momentum = 0.9
weight_decay = 5e-4

# Data transforms (standard for CIFAR10)
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR10
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:07<00:00, 23.9MB/s] 


In [40]:
#Training the model on CIFAR10
model = ResNet18(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, trainloader, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs,feats = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

train_model(model, train_loader)

Epoch 1, Loss: 1.3867
Epoch 2, Loss: 0.9506
Epoch 3, Loss: 0.8059
Epoch 4, Loss: 0.7253
Epoch 5, Loss: 0.6638


In [14]:
def compute_acc(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            logits, _ = model(x)
            preds = torch.argmax(logits, dim=1)  # Pick class index with highest logit
            correct += (preds == y).sum().item()  # Sum of correct predictions
            total += y.size(0)  # Total number of examples

    return correct / total

In [18]:
from torch.utils.data import Subset, DataLoader

def split_dataset_by_class(dataset, forget_class):
    retain_indices = []
    forget_indices = []

    for idx in range(len(dataset)):
        _, label = dataset[idx]
        if label == forget_class:
            forget_indices.append(idx)
        else:
            retain_indices.append(idx)

    retain_subset = Subset(dataset, retain_indices)
    forget_subset = Subset(dataset, forget_indices)
    
    return retain_subset, forget_subset

forget_class = 1

# Use this for both train and test datasets
retain_train_ds, forget_train_ds = split_dataset_by_class(train_loader.dataset, forget_class)
retain_test_ds, forget_test_ds = split_dataset_by_class(test_loader.dataset, forget_class)

# Wrap them in DataLoaders
retain_train_dl = DataLoader(retain_train_ds, batch_size=train_loader.batch_size, shuffle=True, num_workers=2)
forget_train_dl = DataLoader(forget_train_ds, batch_size=train_loader.batch_size, shuffle=True, num_workers=2)

retain_test_dl = DataLoader(retain_test_ds, batch_size=test_loader.batch_size, shuffle=False, num_workers=2)
forget_test_dl = DataLoader(forget_test_ds, batch_size=test_loader.batch_size, shuffle=False, num_workers=2)

In [41]:
print("Evaluating model accuracy:")

acc_retain_train = compute_acc(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = compute_acc(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_retain_test = compute_acc(model, retain_test_dl, device)
print(f"Retain Test Accuracy: {acc_retain_test:.2%}")

acc_forget_test = compute_acc(model, forget_test_dl, device)
print(f"Forget Test Accuracy: {acc_forget_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 78.43%
Forget Train Accuracy: 88.56%
Retain Test Accuracy: 77.62%
Forget Test Accuracy: 89.60%


## Contrastive unlearning

In [42]:
def contrastive_loss(forget_logits,retain_logits):
    logits =   torch.matmul(forget_logits,retain_logits.T)
    
    logits = -logits.mean()

    return logits

import torch
import torch.nn as nn
import torch.nn.functional as F
import random

def contrastive_loss(forget_logits, retain_logits):
    
    logits = torch.matmul(forget_logits, retain_logits.T)  # [Bf, Br]
    return -logits.mean()  # Minimize similarity

def unlearn(model, forget_train_dl, retain_train_dl, device='cuda'):

    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Convert retrain_train_dl into a list so we can randomly sample from it
    retain_batches = list(retain_train_dl)

    for forget_x, forget_y in forget_train_dl:
        forget_x, forget_y = forget_x.to(device), forget_y.to(device)


        sampled_batches = random.sample(retain_batches, k=4)

        for retain_x, retain_y in sampled_batches:
            retain_x, retain_y = retain_x.to(device), retain_y.to(device)
            optimizer.zero_grad()
        
            forget_logits, forget_feats = model(forget_x)
            retain_logits, retain_feats = model(retain_x)
            
            retain_ce_loss = criterion(retain_logits, retain_y)
            contrastive = contrastive_loss(forget_feats, retain_feats)

            final_loss  = 3*contrastive + retain_ce_loss
            
            # Backprop
            final_loss.backward()
            optimizer.step()

In [43]:
import time

start  = time.time()
unlearn(model, forget_train_dl, retain_train_dl)
end = time.time()

print(f'Time Taken for unlearning: {end-start}')

Time Taken for unlearning: 9.668107748031616


In [44]:
print("Evaluating model accuracy:")

acc_retain_train = compute_acc(model, retain_train_dl, device)
print(f"Retain Train Accuracy: {acc_retain_train:.2%}")

acc_forget_train = compute_acc(model, forget_train_dl, device)
print(f"Forget Train Accuracy: {acc_forget_train:.2%}")

acc_retain_test = compute_acc(model, retain_test_dl, device)
print(f"Retain Test Accuracy: {acc_retain_test:.2%}")

acc_forget_test = compute_acc(model, forget_test_dl, device)
print(f"Forget Test Accuracy: {acc_forget_test:.2%}")

Evaluating model accuracy:
Retain Train Accuracy: 79.54%
Forget Train Accuracy: 0.00%
Retain Test Accuracy: 78.90%
Forget Test Accuracy: 0.00%


## 🔍 Unlearning Evaluation Summary

### ✅ Model Accuracy Before Unlearning
| Dataset           | Accuracy |
|-------------------|----------|
| Retain Train      | 78.43%   |
| Forget Train      | 88.56%   |
| Retain Test       | 77.62%   |
| Forget Test       | 89.60%   |

---

### 🧠 Model Accuracy After Unlearning
| Dataset           | Accuracy |
|-------------------|----------|
| Retain Train      | 79.54%   |
| Forget Train      | **0.00%**   |
| Retain Test       | 78.90%   |
| Forget Test       | **0.00%**   |

---

### ⏱️ Time Taken for Unlearning
- **9.67 seconds**

---

### 📌 Key Observations:
- The model **completely forgot** the target class (`forget_class = 3`), as intended.
- Accuracy on **retain data** was preserved or even slightly improved after unlearning.
- The unlearning process was efficient (under 10 seconds).
